In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import random
import os
import pickle
import matplotlib.pyplot as plt
import time
import shutil
import getpass
import multiprocessing
import mygene
from collections import Counter

from subprocess import Popen, PIPE
from threading import Timer
from sklearn.datasets import make_spd_matrix
from scipy.stats import random_correlation

date = str(datetime.date.today())

f"Today is {date}"

'Today is 2023-12-13'

In [2]:
# Set random seed so this part is reproducible
# https://www.random.org/ 2023-08-09
np.random.seed(3866)

In [3]:
TEST = True
CPU = multiprocessing.cpu_count() - 1

In [4]:
pth = '../../data/tcga/TCGA-LUAD-TPM-recount3-2023-02-21.tsv'
exp = pd.read_csv(pth, sep='\t', index_col=0)
exp = np.log2(exp + 1.0)
exp = exp[exp.mean(axis=1) > 1.0]
exp.index = [x.split(".")[0] for x in exp.index.values]

In [5]:
exp

,5b1e3218-61b6-4d10-8815-c1038c739f05,c3257122-09a9-449c-be4a-107703def60c,6a4c997f-2f28-4888-8d4c-c2dc8b108520,dc7450dc-4a7a-48ee-bef1-f5e5b46db020,642c8ec2-4829-4b2b-af17-a6b52a793e26,520354a4-9c4f-4cde-8e73-004e76d92a2e,d138ed54-1cd4-4fd6-b661-0087046f2730,cd5ae438-d578-472e-b7b2-1dbd52d43233,219a9b4a-c6eb-4d2a-aa44-fdb768ab3135,e1b8206b-c670-41f8-bfed-6d446f6b9737,...,e75d87af-dc22-4a00-8dac-ede1d242b3e5,3e400da6-7337-4af6-8ee7-e642de9f0ef0,cb00373b-2f05-44cb-898d-e12b30a4cef6,fb89a665-b6c3-41d8-91fe-6fbc8ad2ccf9,e820fd60-318f-4094-8d28-5305305508f9,f59229d8-3570-4d7a-aed9-4ad6f128c6f9,82f42146-d66d-488e-a7c8-bb338bf5d5fb,ea8e7e8a-ef5b-4f87-b344-657b397aeefc,8690fa0f-9c1c-4d97-813a-3fb56827fbfc,62c55da6-8041-42c4-bea8-62f24c4537fd
ENSG00000278267,2.552323,1.762184,1.893886,3.524616,2.600335,3.325406,3.685579,1.891663,3.467584,1.807679,...,2.309019,3.550695,1.615803,2.053163,2.997171,2.328708,2.971517,2.226081,3.318031,0.875700
ENSG00000227232,4.055304,3.744318,3.883950,4.846409,3.999698,3.922746,4.374609,3.845044,4.352615,3.773062,...,3.745235,4.892424,4.139847,4.409772,4.478548,4.052578,4.397604,4.637211,4.911964,3.711784
ENSG00000268903,2.085083,1.590173,1.818877,3.177096,3.693224,3.001828,3.133968,1.913368,3.579869,2.293987,...,3.795403,3.707855,1.468177,1.913144,1.436922,0.902827,2.546702,1.618587,2.734908,0.379382
ENSG00000269981,4.312792,3.142040,3.691967,4.674231,4.703756,3.974514,4.529779,3.508983,4.902628,3.430295,...,5.139666,5.536816,3.342879,3.920521,3.463861,3.056884,4.862474,3.760362,4.066558,1.765052
ENSG00000239906,2.096936,0.879187,0.965770,1.912791,1.802282,1.649522,2.288712,1.221281,2.388678,1.156045,...,1.687334,2.957816,0.520748,1.111419,0.705018,0.746442,2.031163,0.827368,1.898649,0.434600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000185275,5.153201,5.274613,7.354628,6.560177,7.490239,3.628042,5.179383,3.770000,8.098728,7.073214,...,6.408342,3.765253,6.298575,4.932204,5.422277,5.949543,8.357253,7.058302,6.478372,6.127676
ENSG00000131002,0.000000,1.368638,0.000000,0.000000,0.014483,3.230696,0.000000,0.000000,0.007866,0.000000,...,1.318094,4.371518,0.000000,0.006775,2.629548,0.200506,2.771020,0.000000,1.605947,0.000000
ENSG00000012817,0.000000,2.204699,0.009356,0.000000,0.068357,4.123874,0.000000,0.000000,0.000000,0.008616,...,2.284019,5.408824,0.000000,0.000000,3.874366,0.448162,4.798192,0.000000,2.803687,0.000000
ENSG00000198692,0.000000,2.220330,0.021640,0.000000,0.040088,2.898788,0.000000,0.000000,0.021861,0.000000,...,1.881614,3.895572,0.000000,0.000000,3.495109,0.607704,3.735557,0.000000,2.146682,0.000000


In [6]:
import errno    
import os

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

In [7]:
def cohend(d1, d2):
    """
    https://machinelearningmastery.com/effect-size-measures-in-python/
    """
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

In [8]:
def AlmightyCorrcoefEinsumOptimized(O, P):
    """
    https://github.com/ikizhvatov/efficient-columnwise-correlation/blob/master/columnwise_corrcoef_perf.py
    """
    (n, t) = O.shape      # n traces of t samples
    (n_bis, m) = P.shape  # n predictions for each of m candidates

    DO = O - (np.einsum("nt->t", O, optimize='optimal') / np.double(n)) # compute O - mean(O)
    DP = P - (np.einsum("nm->m", P, optimize='optimal') / np.double(n)) # compute P - mean(P)

    cov = np.einsum("nm,nt->mt", DP, DO, optimize='optimal')

    varP = np.einsum("nm,nm->m", DP, DP, optimize='optimal')
    varO = np.einsum("nt,nt->t", DO, DO, optimize='optimal')
    tmp = np.einsum("m,t->mt", varP, varO, optimize='optimal')
    return cov / np.sqrt(tmp)

In [9]:
def corr2cov(corr, std):
    return corr * np.outer(std, std)

def get_data(genes, # Total universe of genes
             numerators, # Genes with positive association with "response"
             denominators, # Genes with negative associate with "response"
             effect_size, 
             subtype_frac, 
             mu, 
             cov,
             nsamples):
    
    print("Creating test data...")
    
    # Subtype size
    ssize = int(subtype_frac * nsamples)
    # Background size
    bsize = nsamples - ssize
    
    # Samples for background population
    bsamples = ['nonresponders%d' % x for x in range(bsize)]
    # Samples for subtype population
    ssamples = ['responders%d' % x for x in range(bsize, nsamples)]

    # Total number of samples
    samples = bsamples + ssamples
    
    assert len(samples) == nsamples
    
    print("Sampling from TRAIN multivariate normal")
    
    train = pd.DataFrame(index=genes,
                         columns=samples,
                         data=np.random.multivariate_normal(mu, 
                                                            cov, 
                                                            nsamples).T)
    
    print("Sampling from TEST multivariate normal")
    test = pd.DataFrame(index=genes,
                        columns=samples,
                        data=np.random.multivariate_normal(mu, cov, nsamples).T)
    
    print("Randomly sampling DEGs")
    degs = numerators + denominators
    
    print("Randomly sampling covariance matrix")
    _cov = make_spd_matrix(len(degs))
    
    print("Creating subtype mean expression vector")
    subtype_means = []
    
    for i, gene in enumerate(degs):
        
        p1 = (bsize - 1) * (std[gene] ** 2)
        p2 = (ssize - 1) * _cov[i, i]
        
        s = np.sqrt( (p1 + p2) / (nsamples - 2) )
        
        if gene in numerators:
            _subtype_mean = mu[gene] + effect_size * s
            
        else:
            _subtype_mean = mu[gene] - effect_size * s
        
        subtype_means.append(_subtype_mean)
    
    print("Adding TRAIN subtyping mean signal")
    train.loc[degs, ssamples] = np.random.multivariate_normal(subtype_means, _cov, ssize).T
    
    print("Adding TEST subtyping mean signal")
    test.loc[degs, ssamples] = np.random.multivariate_normal(subtype_means, _cov, ssize).T
    
    #for gene in degs[:3] + degs[-3:]:
    #    bins = np.linspace(0, 10, 25)
    #    sns.histplot(train.loc[gene, bsamples], 
    #                 kde=False,
    #                 label='background',
    #                 bins=bins)
    #    sns.histplot(train.loc[gene, ssamples], 
    #                 kde=False,
    #                 label='active',
    #                 bins=bins)
    #    plt.show()
        
    cds = []
    for gene in numerators:
        
        cd = cohend(train.loc[gene, ssamples].values,
                    train.loc[gene, bsamples].values)
        
        cds.append(cd)
        
    return train, test, degs, np.mean(cds)

In [12]:
def validate(in_dir, 
             out_dir, 
             expression, 
             effect_size, 
             nratios, 
             corr_threshold, 
             subtype_frac,
             mu, 
             cov, 
             corr, 
             nsamples=200):
    
    tag = f'eff-{effect_size}-ndiff-{nratios}-frac-{subtype_frac}-corr-{corr_threshold}'
    
    print(tag)
    
    train_pth = os.path.join(in_dir, f'synthetic-train-{tag}-{date}.tsv')
    
    if os.path.exists(train_pth):
        return
    
    test_pth = os.path.join(in_dir, f'synthetic-test-{tag}-{date}.tsv')
    
    deg_pth = os.path.join(in_dir, f'synthetic-degs-{tag}-{date}.tsv')
    
    cohen_pth = os.path.join(in_dir, f'synthetic-cohen-{tag}-{date}.tsv')
    
    # Take the highest correlated genes and assume it is a response signatue 
    # We can experiment with lowering the correlation
    xs, ys = np.where((0.95 > np.triu(corr, k=1)) & (np.triu(corr, k=1) > corr_threshold))

    numerators, denominators = [], []

    xs_counter = Counter(xs)
    ys_counter = Counter(ys)

    potential_pairs = [] 
    for k, v in xs_counter.items():
        if v == 1:
            potential_pairs.append((k, ys[np.where(xs == k)[0][0]]))

    for n, d in potential_pairs[:nratios]:
        
        if n not in numerators:
            numerators.append(expression.index.values[n])

        if d not in denominators:
            denominators.append(expression.index.values[d])
    
    
    train, test, degs, cohen = get_data(mu.index.values, 
                                        numerators,
                                        denominators,
                                        effect_size,
                                        subtype_frac,
                                        mu, 
                                        cov,
                                        nsamples)
    
    train.to_csv(train_pth, sep='\t')
    test.to_csv(test_pth, sep='\t')
    
    with open(deg_pth, 'w') as f:
        f.write('\n'.join(degs))
        
    with open(cohen_pth, 'w') as f:
        f.write(str(cohen))

def run_validation(in_dir: str,
                   out_dir: str,
                   data: pd.DataFrame,
                   effects, 
                   ndiffs,
                   corr_thresholds,
                   subtype_fractions,
                   mu, cov,
                   corr):   
    
    for effect in effects:
        for ndiff in ndiffs:
            for corr_threshold in corr_thresholds:
                for subtype_frac in subtype_fractions:
                    validate(in_dir, 
                             out_dir,
                             data, 
                             effect, 
                             ndiff,
                             corr_threshold,
                             subtype_frac,
                             mu, 
                             cov, 
                             corr)

In [13]:
if TEST:
    in_dir = '../../data/test/test_input/%s' % date
    out_dir = '../../data/test/test_output/%s' % date
    
    if os.path.exists(in_dir):
        shutil.rmtree(in_dir)
        
    if os.path.exists(out_dir):
        shutil.rmtree(out_dir)
        
    mkdir_p(in_dir)
    mkdir_p(out_dir)
    
    _expression = exp.reindex(exp.var(axis=1).sort_values(ascending=False).head(5000).index)
    
    mu = _expression.mean(axis=1)
    std = _expression.std(axis=1)
    corr = AlmightyCorrcoefEinsumOptimized(_expression.T, _expression.T)
    cov = corr2cov(corr, std.values.reshape(len(std), 1))
    
    run_validation(in_dir,
                   out_dir,
                   _expression,
                   [3.0, 2.0, 1.0, 0.75, 0.5, 0.25, 0.10, 0.5], # Effect 
                   [50, 25, 15, 10, 5], # NDiffs
                   [0.25, 0.5, 0.75], # Correlation Thresholds
                   [0.05, 0.10, 0.15, 0.2, 0.25], # Subtype Fractions
                   mu, 
                   cov,
                   corr)

eff-1.0-ndiff-50-frac-0.05-corr-0.25
Creating test data...
Sampling from TRAIN multivariate normal
Sampling from TEST multivariate normal


KeyboardInterrupt: 